In [78]:
library("dplyr")

## Inputs: 

1. Set of nodes *Z*
2. Ordering of the nodes
3. Upper bound on the number of parents (for each node) *u*
4. Database *D* of m cases

In [372]:
N_tensor <- function(D, i, V.i, pi.i){
    result <- D %>% count(D[,pi.i], name='j_counts')
    result <- result$j_counts
        for (k in V.i){
            mask <- D[, i]==k
            k_count <- D[mask,] %>% count(D[mask,pi.i])
            result<-data.frame(result, k_count[,length(k_count)])
            colnames(result)[length(colnames(result))]<- k
        }
        result[is.na(result)] <- 0
    names(result)[1] <- 'j_counts'
    names(result)[2:(length(V.i)+1)] <- V.i
    return(result)
}

g <- function(V.i, r.i, N.tensor){
    N.ij <- N.tensor$j_counts
    q.i <- length(N.ij) # number of unique combinations of the parents realizations
    result <- 1
    for (j in 1:q.i) {
        a <- 1
        for (k in V.i){a <- a*factorial(N.tensor[j,as.character(k)])}
        result <- result*a*factorial(r.i-1)/factorial(N.ij[j]+r.i-1)
    }
    #print(paste('func',result))
    return(result)
}


k2 <- function(Z, u, D){
    V <- unname(sapply(D, unique)) #it's a matrix
    r <- unname(sapply(D, n_distinct)) #it's a vector
    parents <- list()
    for (i in 1:length(Z)){
        cat('\n iteration', i)
        pi.i <- NULL 
        Nijk <- N_tensor(D, Z[i], V[,i], pi.i)
        P_old <- g(V[,i],r[i], Nijk)
        proceed <- TRUE
        while (proceed & (length(pi.i)<u & i>1)){
            pred.i <- setdiff(Z[1:i-1], pi.i)
            #print(paste('pi', pi.i))
            #print(paste('pred', pred.i))
            P <- NULL
            for (node in pred.i){
                new_set <- union(pi.i, node)
                Nijk <- N_tensor(D, Z[i], V[,i], new_set)
                P<- c(P, g(V[,i], r[i], Nijk))
            }
            cat('\n POLD IS ', P_old)
            cat('\n p is ', P, 'n')
            if (max(P) > P_old){
                print("ok")
                max.indx <- which.max(P)
                P_old <- P[max.indx]
                pi.i <- union(pi.i, pred.i[max.indx])
            }
            else {proceed = FALSE}
        }
        append(parents, list(pi.i))
    } 
    return(parents)
}


In [373]:
x1 <- c(1,1,0,1,0,0,1,0,1,0)
x2 <- c(0,1,0,1,0,1,1,0,1,0)
x3<- c(0,1,1,1,0,1,1,0,1,0)

D <- data.frame(x1, x2, x3)

print(D)


   x1 x2 x3
1   1  0  0
2   1  1  1
3   0  0  1
4   1  1  1
5   0  0  0
6   0  1  1
7   1  1  1
8   0  0  0
9   1  1  1
10  0  0  0


In [374]:
papitos <- k2(c(1,2,3), u=3, D)
papitos


 iteration 1
 iteration 2
 POLD IS  0.0003607504
 p is  0.001111111 n[1] "ok"

 POLD IS  0.001111111
 p is  n

Warning message in max(P):
"ningun argumento finito para max; retornando -Inf"


 iteration 3
 POLD IS  0.0004329004
 p is  0.0005555556 0.1333333 n[1] "ok"


ERROR: Error in data.frame(result, k_count[, length(k_count)]): arguments imply differing number of rows: 4, 3


In [210]:
pi.i <- c(2,3)
#D[,pi.i] %>% group_by_all() %>% count()
D %>% count(D[,pi.i], name='j_count')
D %>% count(D[,2], D[,3],name='j_count')

x2,x3,j_count
0,0,4
0,1,1
1,1,5


"D[, 2]","D[, 3]",j_count
0,0,4
0,1,1
1,1,5


In [299]:
?merge

In [295]:
#D[D[, 1]==1,pi.i] %>% group_by_all() %>% summarise(name=n())
pi.i<-c(1,2)
D[D[, 2]==0, ] %>% count(D[D[, 2]==0,pi.i])
#merge(D1, k_count, all.x=TRUE)
#result[result[, 3]==0, ] %>% count(x1, x2)

x1,x2,n
0,0,4
1,0,1


In [121]:
names(D)[1]

[1] "x1"

In [76]:
0:1

[1] 0 1

In [77]:
x <- c(2,3,4,2,1)
unique(x)

[1] 2 3 4 1